In [ ]:
%%capture
!pip install transformers==4.11.3
!pip install -U datasets
!pip install torchaudio
!pip install librosa
!pip install jiwer

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

ERROR:root:HfApi.login: This method is deprecated in favor of `set_access_token`.


Login successful
Your token has been saved to /root/.huggingface/token
Authenticated through git-credential store but this isn't the helper defined on your machine.
You might have to re-authenticate when pushing to the Hugging Face Hub. Run the following command in your terminal in case you want to set this credential helper as the default

git config --global credential.helper store


In [ ]:
from datasets import load_dataset, load_metric

common_voice_train = load_dataset("mozilla-foundation/common_voice_7_0","lg",use_auth_token=True,split='train+validation')
common_voice_test = load_dataset("mozilla-foundation/common_voice_7_0","lg",use_auth_token=True,split='test')

Downloading:   0%|          | 0.00/9.79k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.92k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/49.7k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.72G [00:00<?, ?B/s]

0 examples [00:00, ? examples/s]

0 examples [00:00, ? examples/s]

0 examples [00:00, ? examples/s]

0 examples [00:00, ? examples/s]

0 examples [00:00, ? examples/s]

Dataset common_voice downloaded and prepared to /root/.cache/huggingface/datasets/mozilla-foundation___common_voice/lg/7.0.0/33e08856cfa0d0665e837bcad73ffd920a0bc713ce8c5fffb55dbdf1c084d5ba. Subsequent calls will reuse this data.


In [ ]:
common_voice_train


Dataset({
    features: ['client_id', 'path', 'audio', 'sentence', 'up_votes', 'down_votes', 'age', 'gender', 'accent', 'locale', 'segment'],
    num_rows: 10175
})

In [ ]:
common_voice_train

Dataset({
    features: ['client_id', 'path', 'audio', 'sentence', 'up_votes', 'down_votes', 'age', 'gender', 'accent', 'locale', 'segment'],
    num_rows: 10175
})

In [ ]:
common_voice_train = common_voice_train.remove_columns(["accent", "age", "client_id", "down_votes", "gender", "locale", "segment", "up_votes"])
common_voice_test = common_voice_test.remove_columns(["accent", "age", "client_id", "down_votes", "gender", "locale", "segment", "up_votes"])

In [ ]:
common_voice_train

Dataset({
    features: ['path', 'audio', 'sentence'],
    num_rows: 10175
})

In [ ]:
from datasets import ClassLabel
import random
import pandas as pd
from IPython.display import display, HTML

def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)
    
    df = pd.DataFrame(dataset[picks])
    display(HTML(df.to_html()))

In [ ]:
show_random_elements(common_voice_train.remove_columns(["path","audio"]), num_examples=20)

,sentence
0,"""Omuntu alunda ebisolo aba n'omukwano mungi eri abantu."""
1,Abakulembeze ba disitulikiti balina okugonjoola ekizibu amangu ddala.
2,Kakuyege yabadde wa buvuyo.
3,Poliisi tegumiikiriza bantu batyoboola eddembe lya bannabwe mu kitundu.
4,"""Satifikeeti z'obuzaale bwabwe tezirabikako."""
5,Kiraabu yasanyusa abawagizi baayo bwe yawangula oluzannya.
6,"""Amasomero agali ku musingi gw'ekkanisa galina enteekateka y'okuliisa abaana baago."""
7,"""Abaami n'abakyala abali mu kifo kino nsaba mumpulirizeeko katono."""
8,"""Omusujja gw'ensiri ne lubyamira ndwadde za bulabe."""
9,Abantu balina okwegatta.


In [ ]:
import re
chars_to_ignore_regex = '[\,\?\.\!\-\;\:\"\“\%\‘\”\�]'

def remove_special_characters(batch):
    batch["sentence"] = re.sub(chars_to_ignore_regex, '', batch["sentence"]).lower() + " "
    return batch

In [ ]:
common_voice_train = common_voice_train.map(remove_special_characters)
common_voice_test = common_voice_test.map(remove_special_characters)

  0%|          | 0/10175 [00:00<?, ?ex/s]

  0%|          | 0/4276 [00:00<?, ?ex/s]

In [ ]:
show_random_elements(common_voice_train.remove_columns(["path","audio"]))

,sentence
0,gavumenti eruubirira kumalawo bikolwa by'obumenyi bw'amateeka mu bitundu ebiri ku nsalo
1,waliwo ababbi bangi mu kitundu kyaffe
2,akazannyo kano kannyumira
3,kcca ekoze ekisobooka okukungaanya kasasiro mu kampala
4,poliisi yategeezebwako ku kutiisibwatiisibwa okwali kugenda mu maaso
5,buli muyizi alina okusoma ennyo
6,ntoroko ye disitulikiti esingamu obuli bw'enguzi
7,yawaabidde ababbi ku poliisi
8,abantu abamu beefuulira pulojekiti y'amasanyalaze kubanga omutuuze yasuubizibwa ennyumba
9,omwana baamukutte ng'atoloka ne bamukuba emiggo


In [ ]:
def extract_all_chars(batch):
  all_text = " ".join(batch["sentence"])
  vocab = list(set(all_text))
  return {"vocab": [vocab], "all_text": [all_text]}

In [ ]:
vocab_train = common_voice_train.map(extract_all_chars, batched=True, batch_size=-1, keep_in_memory=True, remove_columns=common_voice_train.column_names)
vocab_test = common_voice_test.map(extract_all_chars, batched=True, batch_size=-1, keep_in_memory=True, remove_columns=common_voice_test.column_names)

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

In [ ]:
vocab_list = list(set(vocab_train["vocab"][0]) | set(vocab_test["vocab"][0]))

In [ ]:
vocab_dict = {v: k for k, v in enumerate(vocab_list)}
vocab_dict

{' ': 0,
 "'": 3,
 '(': 28,
 ')': 25,
 'a': 29,
 'b': 16,
 'c': 10,
 'd': 17,
 'e': 1,
 'f': 7,
 'g': 8,
 'h': 11,
 'i': 5,
 'j': 27,
 'k': 9,
 'l': 12,
 'm': 15,
 'n': 26,
 'o': 2,
 'p': 22,
 'r': 21,
 's': 20,
 't': 4,
 'u': 19,
 'v': 23,
 'w': 24,
 'x': 18,
 'y': 14,
 'z': 6,
 '’': 13}

In [ ]:
vocab_dict["|"] = vocab_dict[" "]
del vocab_dict[" "]

In [ ]:
len(vocab_dict)

30

In [ ]:
del vocab_dict["("]
del vocab_dict[")"]


In [ ]:
vocab_dict

{"'": 3,
 'a': 29,
 'b': 16,
 'c': 10,
 'd': 17,
 'e': 1,
 'f': 7,
 'g': 8,
 'h': 11,
 'i': 5,
 'j': 27,
 'k': 9,
 'l': 12,
 'm': 15,
 'n': 26,
 'o': 2,
 'p': 22,
 'r': 21,
 's': 20,
 't': 4,
 'u': 19,
 'v': 23,
 'w': 24,
 'x': 18,
 'y': 14,
 'z': 6,
 '|': 0,
 '’': 13}

In [ ]:
vocab_dict["[UNK]"] = len(vocab_dict)
vocab_dict["[PAD]"] = len(vocab_dict)
len(vocab_dict)

30

In [ ]:
import json
with open('vocab.json', 'w') as vocab_file:
    json.dump(vocab_dict, vocab_file)

In [ ]:
from transformers import Wav2Vec2CTCTokenizer

tokenizer = Wav2Vec2CTCTokenizer("./vocab.json", unk_token="[UNK]", pad_token="[PAD]", word_delimiter_token="|")

In [ ]:
from transformers import Wav2Vec2FeatureExtractor

feature_extractor = Wav2Vec2FeatureExtractor(feature_size=1, sampling_rate=16000, padding_value=0.0, do_normalize=True, return_attention_mask=True)

In [ ]:
from transformers import Wav2Vec2Processor

processor = Wav2Vec2Processor(feature_extractor=feature_extractor, tokenizer=tokenizer)

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive/')

Mounted at /content/gdrive/


In [ ]:
processor.save_pretrained("/content/gdrive/MyDrive/wav2vec2-large-xlsr-tamil-demo")

In [ ]:
common_voice_train[0]

{'path': '/root/.cache/huggingface/datasets/downloads/extracted/38a26df4350f0cbefb5a50d4018959b95a7ee56e372bfbcd98491bc12d6599c7/cv-corpus-6.1-2020-12-11/ta/clips/common_voice_ta_19171502.mp3',
 'sentence': 'கதிபெற வேண்டும் என்றே '}

In [ ]:
import torchaudio

def speech_file_to_array_fn(batch):
    # speech_array, sampling_rate = torchaudio.load(batch["path"])
    # batch["speech"] = speech_array[0].numpy()
    # batch["sampling_rate"] = sampling_rate
    batch["target_text"] = batch["sentence"]
    return batch

In [ ]:

common_voice_train = common_voice_train.map(speech_file_to_array_fn)
common_voice_test = common_voice_test.map(speech_file_to_array_fn)

  0%|          | 0/10175 [00:00<?, ?ex/s]

  0%|          | 0/4276 [00:00<?, ?ex/s]

In [ ]:
common_voice_train

Dataset({
    features: ['path', 'audio', 'sentence', 'target_text'],
    num_rows: 10175
})

In [ ]:
import librosa
import numpy as np

def resample(batch):
    batch["speech"] = librosa.resample(np.asarray(batch["audio"]['array']), 48_000, 16_000)
    batch["sampling_rate"] = 16_000
    return batch

In [ ]:
common_voice_train = common_voice_train.map(resample, num_proc=1)
common_voice_test = common_voice_test.map(resample, num_proc=1)

  0%|          | 0/10175 [00:00<?, ?ex/s]

  0%|          | 0/4276 [00:00<?, ?ex/s]

In [ ]:
import IPython.display as ipd
import numpy as np
import random

rand_int = random.randint(0, len(common_voice_train)-1)

ipd.Audio(data=np.asarray(common_voice_train[rand_int]["speech"]), autoplay=True, rate=16000)

In [ ]:
rand_int = random.randint(0, len(common_voice_train)-1)

print("Target text:", common_voice_train[rand_int]["target_text"])
print("Input array shape:", np.asarray(common_voice_train[rand_int]["speech"]).shape)
print("Sampling rate:", common_voice_train[rand_int]["sampling_rate"])

Target text: enkola ewa amawulire amatuufu n'okuteebereza ku ki ekiyinza okuddirira 
Input array shape: (68964,)
Sampling rate: 16000


In [ ]:
def prepare_dataset(batch):
    # check that all files have the correct sampling rate
    assert (
        len(set(batch["sampling_rate"])) == 1
    ), f"Make sure all inputs have the same sampling rate of {processor.feature_extractor.sampling_rate}."

    batch["input_values"] = processor(batch["speech"], sampling_rate=batch["sampling_rate"][0]).input_values
    
    with processor.as_target_processor():
        batch["labels"] = processor(batch["target_text"]).input_ids
    return batch

In [ ]:
common_voice_train = common_voice_train.map(prepare_dataset, remove_columns=common_voice_train.column_names, batch_size=8, num_proc=1, batched=True)
common_voice_test = common_voice_test.map(prepare_dataset, remove_columns=common_voice_test.column_names, batch_size=8, num_proc=1, batched=True)

  0%|          | 0/1272 [00:00<?, ?ba/s]

/usr/local/lib/python3.7/dist-packages/numpy/core/_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)


  0%|          | 0/535 [00:00<?, ?ba/s]

In [ ]:
import torch

from dataclasses import dataclass, field
from typing import Any, Dict, List, Optional, Union

@dataclass
class DataCollatorCTCWithPadding:
    """
    Data collator that will dynamically pad the inputs received.
    Args:
        processor (:class:`~transformers.Wav2Vec2Processor`)
            The processor used for proccessing the data.
        padding (:obj:`bool`, :obj:`str` or :class:`~transformers.tokenization_utils_base.PaddingStrategy`, `optional`, defaults to :obj:`True`):
            Select a strategy to pad the returned sequences (according to the model's padding side and padding index)
            among:
            * :obj:`True` or :obj:`'longest'`: Pad to the longest sequence in the batch (or no padding if only a single
              sequence if provided).
            * :obj:`'max_length'`: Pad to a maximum length specified with the argument :obj:`max_length` or to the
              maximum acceptable input length for the model if that argument is not provided.
            * :obj:`False` or :obj:`'do_not_pad'` (default): No padding (i.e., can output a batch with sequences of
              different lengths).
        max_length (:obj:`int`, `optional`):
            Maximum length of the ``input_values`` of the returned list and optionally padding length (see above).
        max_length_labels (:obj:`int`, `optional`):
            Maximum length of the ``labels`` returned list and optionally padding length (see above).
        pad_to_multiple_of (:obj:`int`, `optional`):
            If set will pad the sequence to a multiple of the provided value.
            This is especially useful to enable the use of Tensor Cores on NVIDIA hardware with compute capability >=
            7.5 (Volta).
    """

    processor: Wav2Vec2Processor
    padding: Union[bool, str] = True
    max_length: Optional[int] = None
    max_length_labels: Optional[int] = None
    pad_to_multiple_of: Optional[int] = None
    pad_to_multiple_of_labels: Optional[int] = None

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        # split inputs and labels since they have to be of different lenghts and need
        # different padding methods
        input_features = [{"input_values": feature["input_values"]} for feature in features]
        label_features = [{"input_ids": feature["labels"]} for feature in features]

        batch = self.processor.pad(
            input_features,
            padding=self.padding,
            max_length=self.max_length,
            pad_to_multiple_of=self.pad_to_multiple_of,
            return_tensors="pt",
        )
        with self.processor.as_target_processor():
            labels_batch = self.processor.pad(
                label_features,
                padding=self.padding,
                max_length=self.max_length_labels,
                pad_to_multiple_of=self.pad_to_multiple_of_labels,
                return_tensors="pt",
            )

        # replace padding with -100 to ignore loss correctly
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        batch["labels"] = labels

        return batch

In [ ]:

data_collator = DataCollatorCTCWithPadding(processor=processor, padding=True)

In [ ]:
wer_metric = load_metric("wer")

Downloading:   0%|          | 0.00/1.91k [00:00<?, ?B/s]

In [ ]:
def compute_metrics(pred):
    pred_logits = pred.predictions
    pred_ids = np.argmax(pred_logits, axis=-1)

    pred.label_ids[pred.label_ids == -100] = processor.tokenizer.pad_token_id

    pred_str = processor.batch_decode(pred_ids)
    # we do not want to group tokens when computing the metrics
    label_str = processor.batch_decode(pred.label_ids, group_tokens=False)

    wer = wer_metric.compute(predictions=pred_str, references=label_str)

    return {"wer": wer}

In [ ]:
vocab_dict

{"'": 17,
 '[PAD]': 29,
 '[UNK]': 28,
 'a': 8,
 'b': 28,
 'c': 10,
 'd': 25,
 'e': 27,
 'f': 2,
 'g': 6,
 'h': 18,
 'i': 16,
 'j': 20,
 'k': 29,
 'l': 1,
 'm': 19,
 'n': 23,
 'o': 4,
 'p': 14,
 'r': 13,
 's': 0,
 't': 24,
 'u': 9,
 'v': 12,
 'w': 22,
 'x': 7,
 'y': 21,
 'z': 3,
 '|': 11,
 '’': 5}

In [ ]:
processor.tokenizer

PreTrainedTokenizer(name_or_path='', vocab_size=28, model_max_len=1000000000000000019884624838656, is_fast=False, padding_side='right', special_tokens={'bos_token': '<s>', 'eos_token': '</s>', 'unk_token': '[UNK]', 'pad_token': '[PAD]'})

In [ ]:
from transformers import Wav2Vec2ForCTC

model = Wav2Vec2ForCTC.from_pretrained(
    "lucio/wav2vec2-large-xlsr-kinyarwanda", 
    attention_dropout=0.1,
    hidden_dropout=0.1,
    feat_proj_dropout=0.0,
    mask_time_prob=0.05,
    layerdrop=0.1,
    gradient_checkpointing=True, 
    ctc_loss_reduction="mean", 
    pad_token_id=processor.tokenizer.pad_token_id,
    # vocab_size=len(processor.tokenizer)
)

Downloading:   0%|          | 0.00/1.57k [00:00<?, ?B/s]

/usr/local/lib/python3.7/dist-packages/transformers/configuration_utils.py:337: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  "Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 "


Downloading:   0%|          | 0.00/1.18G [00:00<?, ?B/s]

In [ ]:
model.freeze_feature_extractor()

In [ ]:
model.config.vocab_size=len(processor.tokenizer)+1

In [ ]:
model.config.vocab_size

29

In [ ]:
!mkdir model_files

In [ ]:
from transformers import TrainingArguments

training_args = TrainingArguments(
  output_dir="/content/model_files",
  # output_dir="./wav2vec2-large-xlsr-turkish-demo",
  group_by_length=True,
  per_device_train_batch_size=2,
  gradient_accumulation_steps=2,
  evaluation_strategy="steps",
  num_train_epochs=30,
  fp16=True,
  save_steps=400,
  eval_steps=400,
  logging_steps=400,
  learning_rate=3e-4,
  warmup_steps=500,
  save_total_limit=2,
)

In [ ]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    data_collator=data_collator,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=common_voice_train,
    eval_dataset=common_voice_test,
    tokenizer=processor.feature_extractor,
)

Using amp fp16 backend


In [ ]:
trainer.train("/content/gdrive/MyDrive/checkpoint-17600")

Loading model from /content/gdrive/MyDrive/checkpoint-17600).
***** Running training *****
  Num examples = 10175
  Num Epochs = 30
  Instantaneous batch size per device = 2
  Total train batch size (w. parallel, distributed & accumulation) = 4
  Gradient Accumulation steps = 2
  Total optimization steps = 76320
  Continuing training from checkpoint, will skip to saved global_step
  Continuing training from epoch 6
  Continuing training from global step 17600
  Will skip the first 6 epochs then the first 4672 batches in the first epoch. If this takes a lot of time, you can add the `--ignore_data_skip` flag to your launch command, but you will resume the training on data already seen by your model.


  0%|          | 0/4672 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/transformers/feature_extraction_utils.py:158: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at  ../torch/csrc/utils/tensor_new.cpp:201.)
  tensor = as_tensor(value)
/usr/local/lib/python3.7/dist-packages/transformers/models/wav2vec2/modeling_wav2vec2.py:882: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  return (input_length - kernel_size) // stride + 1


Step,Training Loss,Validation Loss,Wer
18000,1.472700,0.431367,0.598605
18400,0.187000,0.403431,0.584004
18800,0.161100,0.387393,0.597263
19200,0.158100,0.340409,0.571008
19600,0.149000,0.355029,0.572939
20000,0.147400,0.360028,0.568847
20400,0.143800,0.382809,0.568978
20800,0.141000,0.405285,0.583415
21200,0.124100,0.375957,0.592614
21600,0.122100,0.369274,0.594579


***** Running Evaluation *****
  Num examples = 4276
  Batch size = 8
Saving model checkpoint to /content/model_files/checkpoint-18000
Configuration saved in /content/model_files/checkpoint-18000/config.json
Model weights saved in /content/model_files/checkpoint-18000/pytorch_model.bin
Configuration saved in /content/model_files/checkpoint-18000/preprocessor_config.json
/usr/local/lib/python3.7/dist-packages/transformers/models/wav2vec2/modeling_wav2vec2.py:882: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  return (input_length - kernel_size) // stride + 1
***** Running Evaluation *****
  Num examples = 4276
  Batch size = 8
Saving model checkpoint to /content